In [ ]:
##以下均为数据处理流程
f1=open('D:/unilm/data/hg38_mrna.fasta','r')
f2=open('D:/unilm/data/mrna.txt','w')
#a=f1.readlines()
s=0
for lines in f1.readlines():
    if '>' in lines:
        s=s+1
        if s == 1:
            continue
        else:
            f2.write('\n')
            continue
    f2.write(lines.strip('\n'))
f1.close()
f2.close()

In [ ]:
#对输入的fasta格式文件筛选，选取<1500长度的UTR且<6000长度的CDS序列作为数据集
f2=open('D:/unilm/data/mrna.txt','r')
f3=open('D:/unilm/data/mrna_final.txt','w')
for lines in f2.readlines():
    list_5utr=[]
    list_cds=[]
    list_3utr=[]
    y=0
    for strings in lines:
        if strings.isupper():
            list_cds.append(strings)
            y=y+1
            continue
        if y != 0 :
            if strings.islower():
                list_3utr.append(strings)
                continue
        list_5utr.append(strings)                   
    if len(list_5utr) <= 1500 and len(list_5utr) >1 and len(list_cds) <= 6000 and len(list_cds)!=0 and len(list_3utr) <= 1500 and len(list_3utr) != 0 :
        f3.write(lines)
f2.close()
f3.close()

In [ ]:
#检查筛选后序列数量
f3=open('D:/unilm/data/mrna_final.txt','r')
a=0
b=0
c=0
for lines in f3.readlines():
    list_5utr=[]
    list_cds=[]
    list_3utr=[]
    y=0
    for strings in lines:
        if strings.isupper():
            list_cds.append(strings)
            y=y+1
            continue
        if y != 0 :
            if strings.islower():
                list_3utr.append(strings)
                continue
        list_5utr.append(strings)            
    if len(list_5utr) <= 20 :
        a=a+1
    if len(list_cds) <= 6000 and len(list_cds) !=0 :
        b=b+1
    if len(list_3utr) <= 50 :
        c=c+1
print(a,b,c)

In [ ]:
#按照8：1：1划分数据集
import numpy as np
data=open('D:/unilm/data/mrna_final.txt','r').readlines()
np.random.shuffle(data)
train_size=int(0.8*len(data))
valid_size=int(0.1*len(data))
train_data=data[:train_size]
valid_data=data[train_size:train_size+valid_size]
test_data =data[train_size+valid_size:]

In [ ]:
#生成训练集、验证集和测试集
f4=open('D:/unilm/data/train.txt','w')
f5=open('D:/unilm/data/valid.txt','w')
f6=open('D:/unilm/data/test.txt','w')
for lines in train_data:
    f4.write(lines)
for lines in valid_data:
    f5.write(lines)
for lines in test_data:
    f6.write(lines)
f4.close()
f5.close()
f6.close()

In [ ]:
#分别生成CDS，5utr，3utr的标准输入格式，CDS为真值，utr为预测值，后续进行机器翻译任务，CDS为src_file,5utr和3utr为tgt_file
f3=open('D:/unilm/data/valid.txt','r')
f4=open('D:/unilm/data/val_cds.txt','w')
f5=open('D:/unilm/data/val_3utr.txt','w')
f6=open('D:/unilm/data/val_5utr.txt','w')
for lines in f3.readlines():
    lines= lines.strip(' ')
    list_5utr=[]
    list_cds=[]
    list_3utr=[]
    y=0
    str_5utr=''
    str_3utr=''
    for strings in lines:
        if strings.isupper():
            list_cds.append(strings)
            y=y+1
            continue
        if y != 0 :
            if strings.islower():
                list_3utr.append(strings)
                continue
        list_5utr.append(strings)
    f4.write(''.join(list_cds).upper()+'\n')
    f5.write(''.join(list_3utr).upper()+'\n')
    f6.write(''.join(list_5utr).upper())
f3.close()
f4.close()
f5.close()
f6.close()

In [ ]:
##以下均为K-mers词典构建流程（根据K值的不同需要改读取和保存的文件名）
#将txt格式转换为fasta
f1=open('D:/unilm/data/3utr.txt','r')
f2=open('D:/unilm/data/3utr.fasta','w')
for lines in f1.readlines():
    f2.write('>'+'\n')
    f2.write(lines)
f1.close()
f2.close()

#计算K-mers,输出所有子片段的频率
from Bio import SeqIO
def build_kmers(seq, k_size):
    """a function to calculate kmers from seq"""
    kmers = []  # k-mer存储在列表中
    n_kmers = len(seq) - k_size + 1
    
    for i in range(n_kmers):
        kmer = seq[i:i + k_size]
        kmers.append(kmer)
        
    return kmers

from collections import Counter
def summary_kmers(kmers):
    """a function to summarize the kmers"""
    kmers_stat = Counter(kmers)
    return kmers_stat
sum=Counter()
for seq_record in SeqIO.parse('3utr.fasta','fasta'):
    a=summary_kmers(build_kmers(seq_record.seq.upper(),3))
    sum=sum+a
print(sum)

In [ ]:
_3mers=dict(sum)
list1=[]
list2=[]
mersdict={}
for keys in _3mers.keys():
    list1.append(keys)
    list2.append(_3mers[keys])
mersdict['seq']=list1
mersdict['num']=list2
import pandas as pd
df=pd.DataFrame(mersdict)
df.to_csv('D:/unilm/data/3mers.csv')

In [ ]:
import pandas as pd
df=pd.read_csv('D:/unilm/data/k-mers/5utr/8mers.csv')
df_sort=df.sort_values(by=['num'],ascending=[False])
b=df_sort.reset_index(drop=True)
a=b.seq.iloc[:65536]
result=a.to_list()
f1=open('D:/unilm/data/k-mers/3utr/8mers_all.txt','w')
for i in result:
    f1.write(str(i)+'\n')
f1.close()

In [ ]:
##以下均为Rouge评价流程
from rouge import FilesRouge
from rouge import Rouge
def rouge_score(pre_list,true_list):
    rouge = Rouge()
    for i in range(len(pre_list)):
        if pre_list[i]=='' or pre_list==' ':
            pre_list[i]='#'
    rouge_scores = rouge.get_scores(pre_list, true_list,avg=True)
    return rouge_scores
pre_list=open('D:/bert_seq2seq/result/bert_2048_5utr.txt','r').readlines()
true_list=open('D:/bert_seq2seq/data/test/test_5utr_token_new.txt','r').readlines()
rouge_score(pre_list,true_list)

In [ ]:
a=scores['rouge-1']
b=(a['r']+a['p']+a['f'])/3
b=(a['r']+a['p']+a['f'])/3
b

In [ ]:
##以下均为Jaccard评价流程
def jaccard_similarity(set1, set2):
    # 计算两个集合的交集
    intersection = len(set1.intersection(set2))
    # 计算两个集合的并集
    union = len(set1.union(set2))
    # 计算Jaccard系数
    jaccard_coefficient = intersection / union
    return jaccard_coefficient

def load_text_file(file):
        # 将文本拆分为单词，并创建一个包含单词的集合
        word_set=(set(file.split()))
        return word_set
f1=open("D:/bert_seq2seq/result/bert_2048_5utr.txt",'r')
f2=open("D:/bert_seq2seq/data/test/test_5utr_token_new.txt",'r')
document1=f1.readlines()
document2=f2.readlines()
jaccard_list=[]
for i in range(len(document1)):
    jaccard_coefficient = jaccard_similarity(load_text_file(document1[i]),load_text_file(document2[i]))
    jaccard_list.append(jaccard_coefficient)

print("Jaccard系数:", sum(jaccard_list)/len(jaccard_list))
f1.close()
f2.close()

In [ ]:
##以下均为BLEU评价流程
from nltk.translate.bleu_score import corpus_bleu
f1=open("D:/bert_seq2seq/result/bert_2048_3utr.txt",'r')
f2=open("D:/bert_seq2seq/data/test/test_3utr_token_new.txt",'r')
pre_list=f1.readlines()
true_list=f2.readlines()
def bleu_score(pre_list,true_list):
    candidates=[]
    references=[]
    for i in range(len(pre_list)):
        if pre_list[i]=='' or pre_list==' ':
            pre_list[i]='#'
    for x in pre_list:
        x=x.strip('\n').strip(' ')
        candidates.append(list(x.split(' ')))
    for y in true_list:
        y=y.strip('\n').strip(' ')        
        references.append(list(y.split(' ')))
    print(candidates[0],references[0])
    return corpus_bleu(references,candidates) 
bleu_score(pre_list,true_list)

In [ ]:
#k-mers分析（寻找特殊核苷酸片段以供优化参考）（比较原始训练集和生成后排名前10的序列，验证模型的有效性）
codon_file=open('D:/bert_seq2seq/result/bert_2048_5utr.txt','r')
codon_file=codon_file.readlines()
def build_kmers(seq, k_size):
    kmers = []  # k-mer存储在列表中
    n_kmers = len(seq) - k_size + 1
    
    for i in range(n_kmers):
        kmer = seq[i:i + k_size]
        kmers.append(kmer)
        
    return kmers
from collections import Counter
def summary_kmers(kmers):
    kmers_stat = Counter(kmers)
    return kmers_stat
kmers_dic={}
for k in range(3,9):
    sum=Counter()
    for seq in codon_file:
        seq=seq.strip('\n').replace(' ','')
        a=summary_kmers(build_kmers(seq,k))
        sum=sum+a
    kmers_dic['{}'.format(k)]=sum
    
print(kmers_dic)

In [ ]:
#对得到的k-mers进行分别排序，并取出前100放入dataframe
import pandas as pd
df_kmers=pd.DataFrame(columns=['5','6','7','8'])
for k in range(5,9):
    list1=[]
    list2=[]
    mersdict={}
    for keys in kmers_dic['{}'.format(k)]:
        list1.append(keys)      #片段
        list2.append(kmers_dic['{}'.format(k)][keys])    #数量
    mersdict['seq']=list1
    mersdict['num']=list2
    df_tmp=pd.DataFrame(mersdict)
    df_tmp=df_tmp.sort_values(by=['num'],ascending=False)
    df_tmp=df_tmp.reset_index(drop=True)
    df_kmers['{}'.format(k)]=df_tmp['seq'][:100]
print(df_kmers)

In [ ]:
#计算每个k-mers的位置概率，并输出概率矩阵，用以制作motif图
kmers=pd.read_csv('D:/bert_crf-master/kmers_3utr_motif.csv')
print(kmers)
for k in range(5,9):
    k_list=[]
    for pos in range(k):
        pos_list=[]
        pos_dic={}
        for rank in range(100):
            pos_list.append(kmers['{}'.format(k)][rank][pos])
        pos_count=Counter(pos_list)
        pos_dic['G']=pos_count['G']/(pos_count['G']+pos_count['C']+pos_count['A']+pos_count['U'])
        pos_dic['C']=pos_count['C']/(pos_count['G']+pos_count['C']+pos_count['A']+pos_count['U'])
        pos_dic['A']=pos_count['A']/(pos_count['G']+pos_count['C']+pos_count['A']+pos_count['U'])
        pos_dic['U']=pos_count['U']/(pos_count['G']+pos_count['C']+pos_count['A']+pos_count['U'])
        #print(pos_dic)
        k_list.append(pos_dic)
    print(k_list)
    k_df=pd.DataFrame(k_list).T
    k_df.to_csv('{}mers_3utr.csv'.format(k))